In [19]:
import json
from IPython.display import Video
from IPython.display import HTML
import cv2
import os
from tqdm import tqdm
import os.path as osp

In [14]:
def images_to_video(image_folder, video_name='output_video.mp4', fps=24):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png") or img.endswith(".jpg")]
    images.sort()

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width,height))

    for image in tqdm(images):
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

In [15]:
def generate_annotiation_file(in_file, video_name, out_file):
    new_ann_file = {}
    new_ann_file['images'] = []
    new_ann_file['annotations'] = []
    new_ann_file['videos'] = []
    with open(in_file, 'r') as f:
        ann_file = json.load(f)
    new_ann_file['categories'] = ann_file['categories']
    for key in ann_file:
        if key == 'images':
            for i in ann_file[key]:
                if video_name in i['file_name']:
                    new_ann_file['images'].append(i)
        if key == 'videos':
            for i in ann_file[key]:
                if i['file_name'] == video_name:
                    new_ann_file['videos'].append(i)
    
    with open(out_file, 'w') as f:
        json.dump(new_ann_file, f)

In [35]:
video_name = "SNMOT-142"

generate_annotiation_file(
in_file="/scratch/hk3820/ByteTrack/datasets/SoccerNet/annotations/test.json",
video_name=video_name,
out_file=f"/scratch/hk3820/ByteTrack/datasets/SoccerNet/annotations/{video_name}.json",
)

### Video

In [22]:
images_to_video(f"/scratch/hk3820/ByteTrack/datasets/SoccerNet/test/{video_name}/img1/", video_name='output_video.mp4', fps=24)

100%|██████████| 750/750 [01:03<00:00, 11.78it/s]


### Running inference given parameters

In [37]:
!python3 main2.py --exp_name mot17epoch14\
--post --grid_off --new_kf_off\
--test_dataset --result_folder results/allenabled/singles/ \
--dataset soccernet --ann_file_name SNMOT-142 \
--w_assoc_emb 1.25 --aw_param 1

Cache filename:  det_best_ckpt.pkl
Loaded model from cache
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
2023-12-13 14:38:01.065 | INFO     | __main__:main:168 - Initializing tracker for SNMOT-142
2023-12-13 14:39:32.334 | INFO     | __main__:main:182 - Processed 50 frames. Time taken: 91.27. fps: 0.55
2023-12-13 14:41:03.898 | INFO     | __main__:main:182 - Processed 100 frames. Time taken: 182.83. fps: 0.55
2023-12-13 14:41:43.278 | INFO     | __main__:main:182 - Processed 150 frames. Time taken: 222.21. fps: 0.68
2023-12-13 14:42:21.077 | INFO     | __main__:main:182 - Processed 200 frames. Time taken: 260.01. fps: 0.77
2023-12-13 14:43:12.711 | INFO     | __main__:main:182 - Processed 250 frames. Time taken: 311.65. fps: 0.80
2023-12-13 14:44:07.492 | INFO     | __main__:main:182 - Processed 300 frames. Time taken: 366.43. fps: 0.82
2023-12-13 14:45:20.171 | INFO     | __main__:main:182 - Processed 350 frames. Time taken: 439.11. fps: 0.80
2023-

In [38]:
!cd /scratch/hk3820/deep-oc-sort/results/allenabled/singles/SoccerNet-val/mot17epoch14_post/data/ \
&& zip soccernet_singles.zip SNMOT-???.txt

  adding: SNMOT-122.txt (deflated 74%)
  adding: SNMOT-125.txt (deflated 75%)
  adding: SNMOT-142.txt (deflated 74%)


In [39]:
!python /scratch/hk3820/csgi2271_finalproject/tools/evaluate_soccernet_v3_tracking.py \
--TRACKERS_FOLDER_ZIP /scratch/hk3820/deep-oc-sort/results/allenabled/singles/SoccerNet-val/mot17epoch14_post/data/soccernet_singles.zip \
--GT_FOLDER_ZIP gt_test.zip \
--BENCHMARK SNMOT --DO_PREPROC False --SEQMAP_FILE SNMOT-Test.txt --TRACKERS_TO_EVAL test --SPLIT_TO_EVAL test --OUTPUT_SUB_FOLDER eval_results

/scratch/hk3820/deep-oc-sort

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/hk3820/.local/lib/python3.8/site-packages/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : ./temp/gt                     
TRACKERS_FOLDER      : ./temp     